In [59]:
# Setting Up Dependencies
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import gmaps
from config import api_key
import numpy as np

gkey = api_key
gmaps.configure(api_key=gkey)

In [60]:
brewery_id = []

for x in range(100):
    brewery_id.append(np.random.randint(0, 6000))

    brewery_id

unique_brewery = pd.DataFrame({"ID" : brewery_id})

unique_id = list(unique_brewery["ID"].unique())

print(f"Gathered {len(unique_id)} unique Brewery IDs.")

Gathered 99 unique Brewery IDs.


In [64]:
# Retrieving Raw Brewery Data
raw_brewery_data = []
start = time.time()
for x in range(len(unique_id)):
    try:
        url = "https://api.openbrewerydb.org/breweries/" + str(unique_id[x])
        response = requests.get(url).json()
        raw_brewery_data.append(response)
        print(f"({x+1}/{len(unique_id)}) Retrieved Brewery ID {unique_id[x]}...")

    except:
        print(f"({x+1}/{len(unique_id)}) Timeout for Brewery ID {unique_id[x]}... skipping")
        
stop = time.time()
time_elapsed = round((stop - start)/60,2)
print(f"\nRetrieved {len(unique_id)} breweries... Time elapsed: {time_elapsed} minutes.")

(1/99) Retrieved Brewery ID 5978...
(2/99) Retrieved Brewery ID 691...
(3/99) Retrieved Brewery ID 4446...
(4/99) Retrieved Brewery ID 1822...
(5/99) Retrieved Brewery ID 139...
(6/99) Retrieved Brewery ID 5238...
(7/99) Retrieved Brewery ID 5641...
(8/99) Retrieved Brewery ID 991...
(9/99) Retrieved Brewery ID 3866...
(10/99) Retrieved Brewery ID 4385...
(11/99) Retrieved Brewery ID 3914...
(12/99) Retrieved Brewery ID 1786...
(13/99) Retrieved Brewery ID 831...
(14/99) Retrieved Brewery ID 1099...
(15/99) Retrieved Brewery ID 2485...
(16/99) Retrieved Brewery ID 4798...
(17/99) Retrieved Brewery ID 787...
(18/99) Retrieved Brewery ID 742...
(19/99) Retrieved Brewery ID 1680...
(20/99) Retrieved Brewery ID 447...
(21/99) Retrieved Brewery ID 297...
(22/99) Retrieved Brewery ID 5201...
(23/99) Retrieved Brewery ID 5251...
(24/99) Retrieved Brewery ID 658...
(25/99) Retrieved Brewery ID 1713...
(26/99) Retrieved Brewery ID 5956...
(27/99) Retrieved Brewery ID 2259...
(28/99) Retrieved B

In [65]:
# Creating DataFrame
brewery_df = pd.DataFrame(raw_brewery_data)
pd.to_csv("raw_brewery_data.csv")
brewery_df

,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,street,updated_at,website_url
0,brewpub,Astoria,United States,5978,46.190557,-123.835821,Wet Dog Cafe & Brewery,5033256975,97103-4201,Oregon,990 Astor St,2018-08-24T16:05:43.052Z,http://www.wetdogcafe.com
1,brewpub,Santa Cruz,United States,691,36.9595996,-122.0490197,Humble Sea Brewing Co,8314316189,95060-5866,California,820 Swift St,2018-08-23T23:29:40.975Z,http://www.humblesea.com
2,micro,Medford,United States,4446,39.8990962932246,-74.8236255591221,Lower Forge Brewery,6099759532,08055-2414,New Jersey,14 S Main St,2018-08-24T15:36:58.210Z,http://www.lowerforge.com
3,micro,Gainesville,United States,1822,None,None,Blackadder Brewing Company,3523390324,32607-6014,Florida,618 NW 60th St Ste A,2018-08-11T21:36:51.477Z,http://www.blackadderbrewing.com
4,brewpub,Tempe,United States,139,None,None,Gordon Biersch Brewery Restaurant - Tempe,4807360033,85281-2870,Arizona,420 S Mill Ave Ste 201,2018-08-11T21:35:47.880Z,http://www.craftworksrestaurants.com
5,micro,Charlotte,United States,5238,35.2199558631579,-80.8054236526316,Resident Culture Brewing Company LLC,7043331862,28205-5203,North Carolina,2101 Central Ave,2018-08-24T15:42:33.639Z,http://www.residentculturebrewing.com
6,micro,Cincinnati,United States,5641,39.2072785,-84.4746169,"Wiedemann Brewing Co, LLC",5138009560,45217-1217,Ohio,4811 Vine St,2018-08-24T15:45:21.541Z,http://www.wiedemannbeer.com
7,contract,Livermore,United States,991,None,None,Sauced BBQ & Spirits,9259611300,94550-3141,California,2300 1st St Ste 120,2018-08-11T21:36:19.762Z,http://www.saucedbbqandspirits.com
8,contract,Saint Paul,United States,3866,None,None,Joseph Wolf Brewing Company,6127519422,55101-2143,Minnesota,85 7th Pl E Ste 200,2018-08-11T21:38:03.037Z,http://www.josephwolfbrewing.com
9,planning,Southampton,United States,4385,None,None,Brewery In Planning - Philadelphia,,08088-9062,New Jersey,,2018-08-11T21:38:22.326Z,


In [72]:
# Drop rows with "None" Lats/Lngs
brewery_df = brewery_df[brewery_df.latitude.str.contains("None") == False]
print(f"Dropped {len(unique_id) - len(brewery_df)} breweries from DataFrame which did not have coordinates.\nTotal Brewery count: {len(brewery_df)}") 
      

Dropped 36 breweries from DataFrame which did not have coordinates.
Total Brewery count: 63


In [73]:
brewery_loc = brewery_df[['latitude','longitude']].astype(float)
brewery_loc.head()

,latitude,longitude
0,46.190557,-123.835821
1,36.959600,-122.049020
2,39.899096,-74.823626
5,35.219956,-80.805424
6,39.207279,-84.474617


In [74]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(brewery_loc, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))